In [22]:
import os  
import base64
import json
from openai import AzureOpenAI  
from azure.identity import DefaultAzureCredential, get_bearer_token_provider  


In [12]:
endpoint = os.getenv("AZURE_OPENAI_ENDPOINT", "https://magoai-eastus2.openai.azure.com/")  
deployment = os.getenv("DEPLOYMENT_NAME", "o3-mini")  

In [13]:
# Initialize Azure OpenAI Service client with Entra ID authentication
token_provider = get_bearer_token_provider(  
    DefaultAzureCredential(),  
    "https://cognitiveservices.azure.com/.default"  
)  

In [14]:
client = AzureOpenAI(  
    azure_endpoint=endpoint,  
    azure_ad_token_provider=token_provider,  
    api_version="2024-12-01-preview",  
)  

In [39]:
def message(role, content):  
    return {"role": role, "content": 
            [{"type": "text", 
             "text": content}]
    }


In [40]:
chat_prompt = [
    {
        "role": "developer",
        "content": [
            {
                "type": "text",
                "text": "You are an AI assistant that helps people find information."
            }
        ]
    }
] 

In [41]:
messages = []

In [42]:
messages = chat_prompt 
messages.append(message(role="user", content="It takes 2 hours to dry one shirt, how many hours does it take to dry 8 shirts?"))

print(json.dumps(messages, indent=2))



[
  {
    "role": "developer",
    "content": [
      {
        "type": "text",
        "text": "You are an AI assistant that helps people find information."
      }
    ]
  },
  {
    "role": "user",
    "content": [
      {
        "type": "text",
        "text": "It takes 2 hours to dry one shirt, how many hours does it take to dry 8 shirts?"
      }
    ]
  }
]


In [58]:
completion = client.chat.completions.create(  
    model=deployment,  
    messages=messages,
    max_completion_tokens=100000,
    stop=None,  
    stream=False ,

    reasoning_effort="high"
)  
  
print(completion.to_json())  

{
  "id": "chatcmpl-B9kUxj0bAtTUrKNun4J0fNQNuhZ3o",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "It still takes 2 hours. Drying is a process that happens concurrently rather than sequentially – if you can hang all 8 shirts to dry at the same time under the same conditions, then they will all finish drying in 2 hours.\n\nHowever, if you were forced to dry the shirts one after the other (which is usually not the case), then it would take 16 hours. But typically, when drying clothes, you’re drying them all at once.",
        "refusal": null,
        "role": "assistant"
      },
      "content_filter_results": {
        "hate": {
          "filtered": false,
          "severity": "safe"
        },
        "protected_material_code": {
          "filtered": false,
          "detected": false
        },
        "protected_material_text": {
          "filtered": false,
          "detected": false
        },

In [59]:
print( f"response: {completion.choices[0].message.content}") 
print( f"reasoning effort: {completion.usage.completion_tokens_details.reasoning_tokens}")

response: It still takes 2 hours. Drying is a process that happens concurrently rather than sequentially – if you can hang all 8 shirts to dry at the same time under the same conditions, then they will all finish drying in 2 hours.

However, if you were forced to dry the shirts one after the other (which is usually not the case), then it would take 16 hours. But typically, when drying clothes, you’re drying them all at once.
reasoning effort: 768
